# Generating Headlines in English Using Transformers

In today’s fast-paced digital landscape, the ability to create engaging and relevant headlines is essential for capturing audience attention and driving interaction. Headlines are often the first encounter readers have with content, and their effectiveness can significantly influence whether an article is read further. With the increasing volume of content and the need for timely updates, automating the headline generation process has become an important opportunity.

This project aims to address this challenge by utilizing **cutting-edge Transformer architecture**, a powerful machine learning model designed for processing sequences. Unlike traditional models, **Transformers leverage self-attention mechanisms to efficiently handle long-range dependencies and capture intricate relationships within the data**. This approach enables the model to generate coherent and contextually relevant headlines by understanding and integrating the nuances of the input text, ultimately enhancing the automation and quality of headline creation.

## Why Transformers for Headline Generation?

Traditional algorithms for text generation often face challenges in capturing and maintaining long-range dependencies within the text, which can result in headlines that lack coherence or relevance. Transformers, with their self-attention mechanism, are designed to address this issue effectively. Unlike earlier models, **Transformers can analyze and integrate information from different parts of the sequence simultaneously**, allowing them to generate headlines that are not only grammatically sound but also contextually nuanced and relevant. This capability makes Transformers particularly suited for generating compelling and coherent headlines, even over long sequences of text.

## Project Goals

The aim of this project is to develop a **Transformer-based** model **capable of generating high-quality, engaging headlines in English**. By training the model on a comprehensive dataset of existing headlines, we seek to produce headlines that are both accurate and creatively crafted. The Transformer’s advanced architecture will enable the generation of headlines that are contextually rich and impactful. This model is intended to support content creators, journalists, and marketers by offering a powerful tool to quickly generate captivating headlines, ultimately boosting productivity and enhancing content engagement.

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import numpy as np
import os
import string
import random

The `causal_attention_mask` function creates a mask that ensures each word in a sequence can only be influenced by previous words and itself, not by future words. This is crucial for text generation models, as it prevents the model from using future information to make predictions, ensuring that the generated text is coherent and correctly ordered.

In [15]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Creates a causal attention mask to ensure that each token in a sequence only attends to previous and current tokens,
    but not to future tokens. This is crucial for autoregressive models where each token should not be influenced
    by tokens that come after it in the sequence.

    The mask is designed to be applied to the attention weights in self-attention mechanisms, such as those used
    in Transformer models. It prevents information from flowing from future tokens to the current token, ensuring
    that predictions for each token depend only on tokens that precede it.

    Parameters:
    - batch_size (int): The number of sequences in the batch.
    - n_dest (int): The length of the destination sequence (number of tokens in the sequence being processed).
    - n_src (int): The length of the source sequence (typically equal to n_dest in self-attention).
    - dtype (tf.DType): The data type for the mask tensor (e.g., tf.float32, tf.int32).

    Returns:
    - tf.Tensor: A tensor of shape [batch_size, n_dest, n_src] where the upper triangle of the dot product matrix
      is masked out with zeros, and the lower triangle (including the diagonal) is filled with ones. This tensor
      can be used to mask the attention weights in a self-attention mechanism, ensuring that each token attends only
      to earlier tokens and itself, but not to future tokens.

    Example:
    >>> causal_mask = causal_attention_mask(2, 4, 4, tf.float32)
    >>> print(causal_mask)
    <tf.Tensor: shape=(2, 4, 4), dtype=float32, numpy=
    array([[[1., 0., 0., 0.],
            [1., 1., 0., 0.],
            [1., 1., 1., 0.],
            [1., 1., 1., 1.]],

           [[1., 0., 0., 0.],
            [1., 1., 0., 0.],
            [1., 1., 1., 0.],
            [1., 1., 1., 1.]]], dtype=float32)>
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

The `TransformerBlock` class is a fundamental building block in modern deep learning models, especially those designed for handling sequences, like text. Imagine you're building a sophisticated machine that can understand and generate language—like a powerful translator or a smart chatbot. This machine **needs to process sequences of words and understand the relationships between them**. That’s where the TransformerBlock comes in.

In [16]:
class TransformerBlock(layers.Layer):
    """
    A single block of the Transformer model architecture. This block combines multi-head self-attention
    and feed-forward neural networks to process input sequences.

    The TransformerBlock is designed to capture complex dependencies in sequential data by using self-attention
    mechanisms. It also includes feed-forward layers to further process the attention outputs, along with normalization
    and dropout layers to stabilize training and prevent overfitting.

    Attributes:
    - embed_dim (int): The dimension of the embedding space.
    - num_heads (int): The number of attention heads in the multi-head attention mechanism.
    - ff_dim (int): The dimension of the feed-forward network hidden layer.
    - rate (float): The dropout rate applied to the attention and feed-forward layers (default is 0.1).

    Methods:
    - call(inputs): Executes the forward pass of the Transformer block. It applies the multi-head attention, adds
      residual connections, normalizes the outputs, and processes them through a feed-forward network.

    Parameters:
    - inputs (tf.Tensor): Input tensor with shape (batch_size, seq_len, embed_dim). Represents the sequence of embeddings.

    Returns:
    - tf.Tensor: Output tensor with shape (batch_size, seq_len, embed_dim). The processed sequence after attention,
      feed-forward operations, and normalization.

    Example:
    >>> transformer_block = TransformerBlock(embed_dim=64, num_heads=4, ff_dim=128)
    >>> inputs = tf.random.uniform((32, 10, 64))  # Example input tensor with batch_size=32, seq_len=10, embed_dim=64
    >>> output = transformer_block(inputs)
    >>> print(output.shape)
    (32, 10, 64)
    """
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()  # Initializes the parent class (layers.Layer).
        # MultiHeadAttention layer to capture relationships between different positions in the sequence.
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)

        # Feed-forward network with a ReLU activation function followed by a linear layer.
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),  # Dense layer with ReLU activation.
                layers.Dense(embed_dim),  # Dense layer to project back to the embedding dimension.
            ]
        )

        # Layer normalization applied before and after the residual connection.
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)  # First layer normalization.

        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)  # Second layer normalization.

        # Dropout layers to prevent overfitting by randomly setting a fraction of input units to zero.
        self.dropout1 = layers.Dropout(rate)  # Dropout after the attention layer.

        self.dropout2 = layers.Dropout(rate)  # Dropout after the feed-forward network.

    def call(self, inputs):
        """
        Defines the forward pass of the Transformer block.

        Arguments:
        inputs -- The input tensor to the Transformer block.

        Returns:
        The output tensor of the Transformer block after applying attention, dropout, and feed-forward network.
        """
        input_shape = tf.shape(inputs)  # Get the shape of the input tensor.
        batch_size = input_shape[0]  # Number of sequences in the batch.
        seq_len = input_shape[1]  # Length of each sequence.

        # Create a causal attention mask to prevent attending to future tokens.
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)

        # Apply multi-head attention with the causal mask.
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)

        # Apply dropout to the attention output.
        attention_output = self.dropout1(attention_output)

        # Add the input (residual connection) to the attention output and normalize.
        out1 = self.layernorm1(inputs + attention_output)

        # Apply the feed-forward network to the normalized output.
        ffn_output = self.ffn(out1)

        # Apply dropout to the feed-forward network output.
        ffn_output = self.dropout2(ffn_output)

        # Add the normalized output of the feed-forward network to the residual connection and normalize.
        return self.layernorm2(out1 + ffn_output)


At its core, the `TransformerBlock` is designed to help the machine focus on different parts of a sequence when making decisions. For example, if you’re translating a sentence, the model needs to understand which words in the sentence are related to each other, even if they are far apart. The TransformerBlock achieves this using two **key mechanisms: attention and feed-forward processing.**

* **Self-Attention Mechanism:**
Think of this as the block's way of looking at all the words in a sentence and figuring out which words should be paid more attention to. It does this by **calculating how each word relates to every other word in the sequence**. This process is known as self-attention. For instance, in the sentence "The cat sat on the mat," the **TransformerBlock** helps the model understand that "cat" and "mat" are related, even though they are not next to each other.

* **Feed-Forward Network:**
After understanding these relationships, the next step is to further process this **information to make it more useful**. This is where the feed-forward network comes in. **It takes the attention outputs and applies additional transformations to refine the information**. **This step helps in capturing complex patterns and making the final output more precise.**

* **Normalization and Dropout:**
To **ensure the model learns effectively and doesn’t overfit to the training data**, the TransformerBlock includes normalization and dropout layers. Normalization helps in stabilizing the training process by adjusting the outputs to a standard scale. Dropout, on the other hand, randomly "drops" some of the data during training to **prevent the model from becoming too dependent on any specific part of the input.**

In essence, the `TransformerBlock` acts as a smart processor within a larger machine learning model. It helps the model focus on important parts of the input sequence, processes this information in a sophisticated way, and ensures that the learning process is stable and robust. This makes it an essential component in creating models that can understand and generate human-like text, perform translations, or even respond intelligently in conversations.

The `TokenAndPositionEmbedding` class is designed to convert sequences of tokens into meaningful numerical representations for a machine learning model. **This class addresses two key aspects:**

* **Token Representation:** It translates each token in the input sequence into a **dense vector**, known as a token embedding. **This vector captures the semantic information of the token**, allowing the model to interpret its meaning.

* **Positional Information:** To capture the order of tokens within the sequence, it generates **positional embeddings**. These embeddings encode the position of each token, ensuring that the model understands the sequential context and the relative positioning of tokens.

By combining **token embeddings with positional embeddings, the class provides a comprehensive representation of each token that includes both its meaning and its position in the sequence**. This combined representation is crucial for models to process and understand sequences accurately, enhancing their ability to perform tasks that depend on the order and context of the tokens.

In [17]:
class TokenAndPositionEmbedding(layers.Layer):
    """
    A custom layer that combines token embeddings and positional embeddings for sequences.
    This layer is designed to convert input tokens into dense vectors and add positional information
    to each token embedding to capture the order of tokens in a sequence.

    The TokenAndPositionEmbedding layer is crucial for models that process sequential data, such as
    natural language processing models, where understanding the position of each token in the sequence
    is essential for interpreting the context and meaning.

    Attributes:
    - maxlen (int): The maximum length of the input sequences. This determines the size of the positional
      embeddings.
    - vocab_size (int): The size of the vocabulary, which determines the number of possible tokens.
    - embed_dim (int): The dimensionality of the embedding space. Each token and position is mapped to a vector of
      this size.

    Methods:
    - call(x): Applies the token and positional embeddings to the input sequences. It generates embeddings for each
      token and adds positional embeddings to these token embeddings to encode the order of tokens in the sequence.

    Parameters:
    - x (tf.Tensor): Input tensor of shape (batch_size, sequence_length), where each value represents a token index
      in the input sequences.

    Returns:
    - tf.Tensor: Output tensor of shape (batch_size, sequence_length, embed_dim), where each token index in the input
      sequences has been converted into an embedding vector, with positional information added to it.

    Example:
    >>> embedding_layer = TokenAndPositionEmbedding(maxlen=100, vocab_size=5000, embed_dim=64)
    >>> input_seq = tf.constant([[1, 5, 9], [2, 6, 3]])
    >>> output = embedding_layer(input_seq)
    >>> print(output.shape)
    (2, 3, 64)
    """
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()  # Initializes the parent class (layers.Layer).

        # Embedding layer for tokens, maps token indices to dense vectors of size `embed_dim`.
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)

        # Embedding layer for positional encodings, maps position indices to dense vectors of size `embed_dim`.
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        """
        Applies token and positional embeddings to the input tensor.

        Arguments:
        x -- The input tensor containing token indices.

        Returns:
        The tensor after adding token and positional embeddings.
        """
        maxlen = tf.shape(x)[-1]  # Get the length of the sequences from the input tensor shape.

        # Generate position indices from 0 to maxlen - 1.
        positions = tf.range(start=0, limit=maxlen, delta=1)

        # Apply the positional embedding layer to position indices.
        positions = self.pos_emb(positions)

        # Apply the token embedding layer to the input tensor.
        x = self.token_emb(x)

        # Add the token embeddings and positional embeddings.
        return x + positions

## Data Reading and Preprocessing

### Data Loading

In [18]:
batch_size = 128

# Path to the dataset file
file = "/content/dataset.txt"

# Check if the file exists
if os.path.exists(file):
    # Create a dataset from the text file
    text_ds = tf.data.TextLineDataset(file)
    text_ds = text_ds.shuffle(buffer_size=256)
    text_ds = text_ds.batch(batch_size)
else:
    # Raise a FileNotFoundError with a descriptive message
    raise FileNotFoundError(f"The file at {file} does not exist.")

The `custom_standardization` function is designed to prepare text data for further processing by cleaning it up in a few key ways. Imagine you’re working with a collection of text that needs to be uniformly formatted to ensure consistency before feeding it into a machine learning model.



In [19]:
def custom_standardization(input_string):
    """Remove html line-break tags and handle punctuation"""
    lowercased = tf.strings.lower(input_string)
    stripped_html = tf.strings.regex_replace(lowercased, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"([{string.punctuation}])", r" \1")

Here’s how the function does this:

* **Lowercasing:** First, the function converts all characters in the text to lowercase. This step is crucial because it ensures that the text is uniform—"Hello" and "hello" will be treated as the same word. **This simplification helps the model handle text more effectively without being thrown off by differences in capitalization.**

* **Removing HTML Line-Break Tags:** Next, the function targets HTML line-break tags (<br />) which often appear in web-scraped text or HTML content. These tags are meant to indicate a new line, but they don’t carry meaningful content for text analysis. The function replaces these tags with spaces to ensure that line breaks don’t disrupt the flow of the text.

* **Handling Punctuation:** Finally, the function deals with punctuation marks. Punctuation can sometimes create issues in text processing, especially if it’s not uniformly handled. The function identifies any punctuation marks and ensures they are surrounded by spaces. This helps in keeping punctuation separate from the words, making it easier to analyze and process the text.

In essence, `custom_standardization` prepares raw text by making it consistently formatted and free of unnecessary HTML tags and improperly handled punctuation. This step is fundamental in ensuring that the text is clean and uniform, setting the stage for more effective and accurate analysis or modeling.

In [20]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 80  # Max sequence size

# Create a vectorization layer and adapt it to the text
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

In preparing text data for a machine learning model, this code snippet is setting up a crucial tool for **transforming raw text into a format that the model can use.**

First, it defines the **scope of the vocabulary**, choosing to focus on the top 20,000 most frequently occurring words. This helps keep the model’s vocabulary manageable and relevant, filtering out less common terms that might clutter the data. It also establishes that each sequence of text will be no longer than 80 tokens, ensuring that all inputs are of a consistent size.

To turn the text into a numerical format, the code creates a TextVectorization layer. This layer will process the text by first cleaning it up using the custom_standardization function, which handles things like removing unnecessary HTML tags and punctuation. Then, it converts the text into integer indices, where each word is mapped to a unique number based on its frequency.

The output sequences are slightly longer than the maximum length specified—set to 81 tokens—to accommodate additional training needs, like predicting the next token in a sequence.

In [21]:
# Preparation of the dictionary/vocabulary
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()  # To get words back from token indices

After setting up the text vectorization layer, the next step is to prepare the vocabulary that the model will use.

This snippet begins by adapting the **vectorize_layer** to the text data. This process involves analyzing the text dataset to build a vocabulary based on the most frequent words. Essentially, the layer learns which words are present and how often they occur.

Once this adaptation is complete, the code retrieves the vocabulary using **vectorize_layer.get_vocabulary()**. This vocabulary is a list where each word is associated with a unique index. This step is crucial because it allows you to convert these indices back into words when interpreting the model's output or debugging the results.

In [22]:
def prepare_lm_inputs_labels(text):
    """
    Shift word sequences by 1 position so that the target for position (i) is
    word at position (i+1). The model will use all words up till position (i)
    to predict the next word.
    """
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

The `prepare_lm_inputs_labels` function is designed to prepare text data for training a language model by creating input-output pairs that reflect how the model will learn to predict the next word in a sequence.

**Here’s how it works:**

* **Expanding Dimensions:** The function starts by adding an extra dimension to the text data. This adjustment is necessary for compatibility with the vectorization layer, which expects input in a specific shape.

* **Tokenization:** Next, the text is converted into numerical indices using the vectorize_layer. This process transforms each word in the text into its corresponding token ID, creating a sequence of integers.

* **Creating Input-Output Pairs:** The function then prepares two key components:

* **Inputs (x):** It takes all tokens except the last one in each sequence. These tokens serve as the input for the model.
* **Labels (y):** It shifts the sequence by one position to the right, so each token in the sequence corresponds to the next word in the original text. These tokens act as the target output the model should learn to predict.

In essence, this function arranges the text data into sequences where the model learns to predict the next word based on the words it has seen so far. This setup is fundamental for training a language model to generate coherent and contextually appropriate text.

In [23]:
text_ds = text_ds.map(prepare_lm_inputs_labels)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

## Building the Transformer Model

In [24]:
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model():
    inputs = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam", loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model

This code sets up a **Transformer model for sequence processing**. It begins by defining key parameters: **embedding size (embed_dim)**, **number of attention heads (num_heads)**, and **hidden layer size (feed_forward_dim)**.

The `create_model` function builds the model with an input layer for token sequences, an embedding layer to convert tokens into dense vectors with positional encodings, and a **TransformerBlock** to capture relationships within the sequence. **The output is then passed through a dense layer to generate predictions.**

The model is compiled with the **Adam optimizer** and a loss function for classification, allowing it to learn and predict based on the input data.

## Monitoring Training with a Custom Text Generation Callback

In [25]:
class TextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"generated text:\n{txt}\n")


# Tokenize starting prompt
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

start_prompt = "Blockchain"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 40
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)

The `TextGenerator` class is a custom callback designed to generate text from a trained model during the training process. It operates in a few key steps:

1. **Initialization:** Upon initialization, the class takes several parameters:

- **max_tokens:** Number of tokens to generate following the prompt.
- **start_tokens:** Initial tokens from which text generation starts.
- **index_to_word:** A mapping from token indices to actual words.
- **top_k:** Limits sampling to the top k token predictions.
- **print_every:** Frequency (in epochs) at which to print the generated text.
- **Text Sampling:** The sample_from method selects the next token based on the model's predicted probabilities. It focuses on the top k most likely tokens and samples from them to ensure diversity in the generated text.

2. **Token Conversion:** The detokenize method converts token indices back into readable words using the `index_to_word` mapping.

3. **Text Generation:** The on_epoch_end method generates text at the end of specified epochs. It continuously predicts and appends new tokens based on the starting prompt and previously generated tokens until the desired number of tokens is reached. The generated text is then printed out.

In essence, this callback provides a way to observe and evaluate the progress of text generation during model training by periodically printing out newly generated text samples.

## Training the Model with Real-Time Text Generation Monitoring

In [ ]:
model = create_model()
model.fit(text_ds, verbose=2, epochs=100, callbacks=[text_gen_callback])

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

In this section, the code sets up and begins training the text generation model. First, the `create_model()` function is called to instantiate and configure the model according to the specified architecture and parameters.

Once the model is ready, the `model.fit` method is used to train it. The training process involves feeding the text_ds dataset into the model. This dataset has been preprocessed and tokenized to prepare it for training. The training runs for 100 epochs, which means the model will learn from the entire dataset 100 times.

To keep track of the model’s progress and assess its performance during training, a custom callback, `text_gen_callback`, is employed. This callback generates text samples at specified intervals, providing real-time feedback on how well the model is learning to generate coherent and relevant text. The verbose=2 setting ensures that detailed information about the training process is logged, helping to monitor and evaluate the model’s performance closely.